# `Step 9: Extraction of Cell Type  & Gene Marker with Gene Expression Level | ` 
- 1-) `True Labels`

# 0- `Config`


In [ ]:
try: 
    if manager == 1: 
        print("Manager Mode: ON")
except:
    print("Manager Mode: OFF")
    %run s0_config.ipynb 

# 1- `Load` 

## 1-) `Data`

In [ ]:
#check_point_s1    = "checkpoint/session_s1.h5ad"
#check_point_s2    = "checkpoint/session_s2.h5ad"
#check_point_s3    = "checkpoint/session_s3.h5ad"
#check_point_s4    = "checkpoint/session_s4.h5ad"
#check_point_s5    = "checkpoint/session_s5.h5ad"
#check_point_s6    = "checkpoint/session_s6.h5ad"
#check_point_s7    = "checkpoint/session_s7.h5ad"
#check_point_s8    = "checkpoint/session_s8.h5ad"
#check_point_s9    = "checkpoint/session_s9.h5ad"

adata = sc.read(check_point_s8)   

In [ ]:
pd.DataFrame(adata.obs["cell_type"].value_counts() )

## 2-) `Variables`
- None

In [ ]:
import_name = check_point_s8_variables

with open( access_data_path(target_folder = f"process_0/checkpoint", target_file = import_name + ".pkl" ) , 'rb') as file:
    Variables_Dict_s8 = pickle.load(file)   

target_gene_number      = Variables_Dict_s8['target_gene_number']

selected_obs_cluster_key          = Variables_Dict_s8['selected_obs_cluster_key'] 
selected_uns_ranked_key           = Variables_Dict_s8['selected_uns_ranked_key']
selected_cluster_names            = Variables_Dict_s8['selected_cluster_names'] 
selected_sample_cluster_barcode   = Variables_Dict_s8['selected_sample_cluster_barcode'] 

true_label_obs_cluster_key          = Variables_Dict_s8['true_label_obs_cluster_key'] 
true_label_uns_ranked_key           = Variables_Dict_s8['true_label_uns_ranked_key']
true_label_cluster_names            = Variables_Dict_s8['true_label_cluster_names'] 
true_label_sample_cluster_barcode   = Variables_Dict_s8['true_label_sample_cluster_barcode'] 

In [ ]:
Variables_Dict_s8['target_gene_number']

In [ ]:
true_label_cluster_names

## 3-) `Functions`
- Expression
- Profile

In [ ]:
def top_marker_genes(adata, subject_obs_cluster_key, subject_uns_ranked_key, target_gene_number): 
    # Collect top markers for each cell type
    subject_topmarker_genes = {}
    for cell_type in adata.obs[subject_obs_cluster_key].unique():
        markers = adata.uns[subject_uns_ranked_key]['names'][cell_type][:target_gene_number]  # Adjust the number as needed
        subject_topmarker_genes[cell_type] = markers
    subject_topmarker_genes_DF = pd.DataFrame(subject_topmarker_genes)
    return subject_topmarker_genes_DF 

In [ ]:
def create_Cluster_Dicts_Calculations(target_gene_number, subject_cluster_names, subject_obs_cluster_key, subject_uns_ranked_key, report): 
    
    subject_Total_DF = pd.DataFrame() 
    for i in subject_cluster_names:
        [print(f"Cluster: {i}") if report else None] 
        [print("-----------------") if report else None] 
        Cell_type_Gene_Gene_expression_Dict, Full_Profile_Dict, Cell_Gene_DF = create_Cluster_Dicts( 
                                                                            target_gene_number      = target_gene_number,
                                                                            current_cluster_barcode = str(i), # subject_sample_cluster_barcode,
                                                                            selected_cluster_key    = subject_obs_cluster_key,
                                                                            selected_ranked_key     = subject_uns_ranked_key,
                                                                            report = report )

        subject_Total_DF = pd.concat([subject_Total_DF, Cell_Gene_DF]) 

    # Modification 
    subject_Total_DF['Expression_Gene'] = subject_Total_DF.apply(lambda row: [row['Expression'], row['Gene']], axis=1)

    return subject_Total_DF, Cell_type_Gene_Gene_expression_Dict, Full_Profile_Dict 

In [ ]:
def create_combination_DF( subject_Total_DF ): 
    new_column_names = pd.unique(subject_Total_DF["Cluster"])  
    new_gene_dict = {} 
    for i in range(len(subject_Total_DF)) :
        current_cluster    = subject_Total_DF["Cluster"].iloc[i] 
        current_gene       = subject_Total_DF["Gene"].iloc[i] 
        current_expression = subject_Total_DF["Expression"].iloc[i] 
        new_gene_dict[current_cluster] = new_gene_dict.get(current_cluster, []) + [[current_expression, current_gene ]] 

    true_label_Gene_Expression_DF = pd.DataFrame(new_gene_dict) 
    # Return 
    return true_label_Gene_Expression_DF 

In [ ]:
operation = access_data_path(target_folder = f"process_0/functions/{folder_s9_extract}", target_file = "tool.ipynb")     
%run $operation 

---
# 3- `Bench`

## 0- `Overview`

In [ ]:
adata

#### 1- `Operation`

In [ ]:
true_data             = access_data_path("process_0/data", "rna_single_cell_type.tsv") 
ensembl_to_symbol     = pd.read_csv( true_data, sep='\t' ) 

unique_ensembl_to_symbol = ensembl_to_symbol[['Gene', 'Gene name']].drop_duplicates('Gene')
unique_ensembl_to_symbol['Gene'] = unique_ensembl_to_symbol['Gene'].str.strip()
unique_ensembl_to_symbol['Gene name'] = unique_ensembl_to_symbol['Gene name'].str.strip()
unique_ensembl_to_symbol.set_index('Gene', inplace=True) 

unique_ensembl_to_symbol_DF = unique_ensembl_to_symbol.reset_index() 
unique_ensembl_to_symbol_DF.columns = ["ensembl_id", "gene_symbol"]

ensembl_to_symbol_map = unique_ensembl_to_symbol_DF.set_index('ensembl_id')['gene_symbol'].to_dict()
ensembl_to_symbol_map

#### 1-) `Selected`


In [ ]:
selected_uns_ranked_key

In [ ]:
selected_top_marker_genes_DF = top_marker_genes( adata = adata, 
                 subject_obs_cluster_key = selected_obs_cluster_key, 
                 subject_uns_ranked_key  = selected_uns_ranked_key, 
                 target_gene_number      = target_gene_number) 

selected_top_marker_genes_DF

In [ ]:
for col in selected_top_marker_genes_DF.columns:
    selected_top_marker_genes_DF[col] = selected_top_marker_genes_DF[col].map(
        lambda x: ensembl_to_symbol_map.get(x, x)
    )

selected_top_marker_genes_DF

##### 2- `Advanced`


In [ ]:
adata.var

In [ ]:
selected_Total_DF, selected_Dict, selected_Profile_Dict  = create_Cluster_Dicts_Calculations(
                                                        target_gene_number = target_gene_number, 
                                                        subject_cluster_names = selected_cluster_names, 
                                                        subject_obs_cluster_key = selected_obs_cluster_key, 
                                                        subject_uns_ranked_key = selected_uns_ranked_key,
                                                        report = False) 
selected_Total_DF

In [ ]:
selected_Total_DF['Gene'] = selected_Total_DF['Gene'].map(ensembl_to_symbol_map)
selected_Total_DF

In [ ]:
selected_Combination_DF = create_combination_DF( subject_Total_DF = selected_Total_DF)
selected_Combination_DF.head(5)

#### 2-) `True Label`


##### 1- `Basic`


In [ ]:
adata.obs["cell_type"]

In [ ]:
true_label_top_marker_genes_DF = top_marker_genes( adata = adata, 
                 subject_obs_cluster_key = true_label_obs_cluster_key, 
                 subject_uns_ranked_key  = true_label_uns_ranked_key, 
                 target_gene_number      = target_gene_number) 

true_label_top_marker_genes_DF 

In [ ]:
for col in true_label_top_marker_genes_DF.columns:
    true_label_top_marker_genes_DF[col] = true_label_top_marker_genes_DF[col].map(
        lambda x: ensembl_to_symbol_map.get(x, x)
)
true_label_top_marker_genes_DF

##### 2- `Advanced`


In [ ]:
true_label_Total_DF, true_label_Dict, true_label_Profile_Dict  = create_Cluster_Dicts_Calculations(
                                                        target_gene_number = target_gene_number, 
                                                        subject_cluster_names = true_label_cluster_names, 
                                                        subject_obs_cluster_key = true_label_obs_cluster_key, 
                                                        subject_uns_ranked_key = true_label_uns_ranked_key,
                                                        report = False) 
true_label_Total_DF

In [ ]:
true_label_Total_DF.to_excel("true_label_Total_DF.xlsx", engine='xlsxwriter') 

In [ ]:
true_label_Total_DF['Gene'] = true_label_Total_DF['Gene'].map( lambda x: ensembl_to_symbol_map.get(x, x) ) 
true_label_Total_DF

In [ ]:
import numpy as np 
true_label_Total_DF["Expression"] = np.round(true_label_Total_DF["Expression"] , 3)

##### 3- `Combination`


In [ ]:
true_label_Combination_DF = create_combination_DF( subject_Total_DF = true_label_Total_DF)
true_label_Combination_DF

In [ ]:
true_label_Combination_DF.to_excel("true_label_Combination_DF.xlsx", engine='xlsxwriter')


## 4-) `Send Extracted Ranked Genes`


#### 1-) `True Label`


In [ ]:
true_label_cell_type_gene_profile      = "true_label_cell_type_gene_profile"
true_label_cell_type_gene_profile_path = access_data_path(target_folder = f"process_0/data", target_file = true_label_cell_type_gene_profile + ".csv" ) 
true_label_cell_type_gene_profile_path 

In [ ]:
true_label_Total_DF.to_csv(true_label_cell_type_gene_profile_path) 


In [ ]:
export_name = "true_label_Total_DF" 
with open(os.path.join(access_data_path(f"{process_barcode}/data/{folder_s9_extract}", f"{export_name}" + ".pkl"))  , 'wb') as file: 
    pickle.dump(true_label_Total_DF, file)   

In [ ]:
export_name = "true_label_top_marker_genes_DF"  
with open(os.path.join(access_data_path(f"{process_barcode}/data/{folder_s9_extract}", f"{export_name}" + ".pkl"))  , 'wb') as file: 
    pickle.dump(true_label_top_marker_genes_DF, file)   

In [ ]:
export_name = "true_label_Combination_DF"  
with open(os.path.join(access_data_path(f"{process_barcode}/data/{folder_s9_extract}", f"{export_name}" + ".pkl"))  , 'wb') as file: 
    pickle.dump(true_label_Combination_DF, file)    

#### 2-) `Selected`


In [ ]:
selected_cell_type_gene_profile      = "selected_cell_type_gene_profile"
selected_cell_type_gene_profile_path = access_data_path(target_folder = f"process_0/data", target_file = selected_cell_type_gene_profile + ".csv" ) 
selected_cell_type_gene_profile_path 

In [ ]:
selected_Total_DF.to_csv(selected_cell_type_gene_profile_path)  

In [ ]:
export_name = "selected_Total_DF" 
with open(os.path.join(access_data_path(f"{process_barcode}/data/{folder_s9_extract}", f"{export_name}" + ".pkl"))  , 'wb') as file: 
    pickle.dump(selected_Total_DF, file)   

In [ ]:
export_name = "selected_top_marker_genes_DF" 
with open(os.path.join(access_data_path(f"{process_barcode}/data/{folder_s9_extract}", f"{export_name}" + ".pkl"))  , 'wb') as file: 
    pickle.dump(selected_top_marker_genes_DF, file)   

In [ ]:
export_name = "selected_Combination_DF"  
with open(os.path.join(access_data_path(f"{process_barcode}/data/{folder_s9_extract}", f"{export_name}" + ".pkl"))  , 'wb') as file: 
    pickle.dump(selected_Combination_DF, file)    

---

---
# `Step 9: Checkpoint` 

#### Operation: Checkpoint (Save) 
Save the result.

In [ ]:
adata.write(check_point_s9)

In [ ]:
adata

In [ ]:
Variables_Dict_s9 ={"target_gene_number"      : target_gene_number, 
                    
                    "selected_obs_cluster_key"           : selected_obs_cluster_key,
                    "selected_uns_ranked_key"            : selected_uns_ranked_key,
                    "selected_cluster_names"             : list(pd.unique(adata.obs[selected_obs_cluster_key])), 
                    "selected_sample_cluster_barcode"    : list(pd.unique(adata.obs[selected_obs_cluster_key]))[0], 
                    

                    "true_label_obs_cluster_key"           : true_label_obs_cluster_key,
                    "true_label_uns_ranked_key"            : true_label_uns_ranked_key,
                    "true_label_cluster_names"             : list(pd.unique(adata.obs[true_label_obs_cluster_key])), 
                    "true_label_sample_cluster_barcode"    : list(pd.unique(adata.obs[true_label_obs_cluster_key]))[0], 
                     }

export_name = check_point_s9_variables

with open( access_data_path(target_folder = f"process_0/checkpoint", target_file = export_name + ".pkl" )  , 'wb') as file:
    pickle.dump(Variables_Dict_s9, file)
